# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

### Imports

In [ ]:
%%capture
# %%capture suppresses jupyter output for this cell
# make sure required packages are installed -- before imports
# udacity installed version is 0.4.1; so this forces upgrade to eliminate env. issues
!pip install torch==1.0.0  
!pip install torchsummary
!pip install seaborn
!pip install pillow
!pip install wget

In [ ]:
# Imports here
import torch                    # version 0.4.1 for Udacity
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR

from torch.utils.data import DataLoader
from torch.utils.data.sampler import RandomSampler 
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision                       # ACTION - do we need all items?
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision import datasets         # ACTION - is this needed?  I think not since we are importing folder of images
from torchvision.datasets import ImageFolder

from torchsummary import summary

import numpy as np
import pandas as pd

# visualization imports
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")

# various utilities
import json
import os
from collections import Counter  # for counting images and histograms
import random
import gc                        # python memory garbage collection, to stay clean
import wget                      # retrieving source data
import zipfile

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# for unpacking `state_dict`
from collections import OrderedDict 

# PIL image processing
from PIL import Image

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

# global setting to use gpu or cpu based on available hardware
if not train_on_gpu:
    device = torch.device('cpu')
    print('\nCUDA is NOT available.  Training on CPU ...')
else:
    device = torch.device('cuda')
    print('CUDA is available!  Training on GPU ...')
    
# setup slack notifications; required SLACK vars in your env variables
import os
from slackclient import SlackClient
token = os.environ['SLACK_API_TOKEN']
channel_id = os.environ['SLACK_CHANNEL_ID']

import datetime
import json




In [ ]:
# confirm token exists in current system
'SLACK_API_TOKEN' in os.environ


In [ ]:
# confirm channel ID exists in current system 
'SLACK_CHANNEL_ID' in os.environ

In [ ]:
sc = SlackClient(token)

def slack_message(message, attachments, channel_id):
    sc = SlackClient(token)
    sc.api_call('chat.postMessage', 
                channel=channel_id, 
                text=message,
                attachments = attachments,
                username='surfaceowl',
                icon_emoji=':robot_face:')


In [ ]:
date = datetime.datetime.now().strftime("%x")
day_of_week = datetime.datetime.now().strftime("%A")

# custom_message = f"manual test of slackclient in jupyter notebook sent on { day_of_week }, { date}"
# slack_message(custom_message, [], channel_id)

In [ ]:
### Global variables - required by Udacity

# ACTION - LIST ASSUMPTIONS IN MARKDOWN CELL, EARLY
num_classes = 102          # we calc this locally, but source data not provided in udacity
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

# global variables
global_saved_file_name = 'model_flowers.v60.pt'

# create manual flag to force to run on cpu
# helps us pretest inference on local machine cpu which may have a gpu
# because Udacity environment is cpu only (avoids surprises on submit)
force_train_on_cpu = False

# number of subprocesses for data loading; use almost all available cpu cores
num_workers = os.cpu_count() -1


# machine learning variables
batch_size = 64

# scheduler - parameters to change learning rate over epochs
milestones = [20, 50, 75]
gamma = 0.1

# future user inputs
learning_rate = 0.001

hidden_layers = "Manual" # choose [Zero, Manual, Auto]
# experiment on parameterizing number of hidden layers
hidden_layers1 = [2208, 512]
hidden_layers2 = [512, 256]
hidden_layer_sizes = list(zip(hidden_layers1, hidden_layers2))
print(list(hidden_layer_sizes))


n_epochs = 30

# ACTION - clean up available models; requires manual investigation of last layer
available_models = {"vgg16":25088,
                    "densenet121" : 1024,
                    "densenet161": 2208,
                    "resnet152" : 2048 }

chosen_model = "densenet161"


# image processing
image_size_required = 224
image_size = 224
mean_train_imgs = [0.485, 0.456, 0.406]
std_train_imgs = [0.229, 0.224, 0.225]

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). You can [download the data here](https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip). The dataset is split into two parts, training and validation. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. If you use a pre-trained network, you'll also need to make sure the input data is resized to 224x224 pixels as required by the networks.

The validation set is used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks available from `torchvision` were trained on the ImageNet dataset where each color channel was normalized separately. For both sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.

### TODO: load data, 

In [ ]:
# define source data directories, use os.path.join to avoid environment path issues
data_dir = os.path.join(os.getcwd(), "flower_data")
train_dir = os.path.join(data_dir, "train")
valid_dir = os.path.join(data_dir, "valid")
test_dir = os.path.join(data_dir, "test")

# check if source data exists
data_dir_exists = os.path.isdir(data_dir)  # returns True/False

# if source data does not exist, download source file - then manually unzip 
if not data_dir_exists:
    sourcedata_url = 'https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip'
    filename = wget.download(sourcedata_url)

# check if test data provided; make a variable for conditional checks below
test_data_exists = os.path.isdir(test_dir)
print("test_data_exists: ", test_data_exists)  # returns True/False

In [ ]:
os.path.join(data_dir, "train")

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# print initial cat_to_name; then sort - easier to understand  categories
print(list(cat_to_name.items())[:10])
cat_to_name = list((int(key), value) for (key,value) in cat_to_name.items())

num_classes = len(cat_to_name)
print("number of classes = ", num_classes)


In [ ]:
# create dataframe mapping directories->classes & counting images
class_matrix = pd.DataFrame(columns={"dir_num", "flower_name", "num_images"})

map_sorted = []
image_stats = []

for index, category in enumerate(cat_to_name):
    # summarize # of images by directory/flower
    dir_num = int(cat_to_name[index][0])
    flower_name = cat_to_name[index][1]
    target_directory = os.path.join(train_dir, str(dir_num))
    num_images = int(len(os.listdir(target_directory)))
    row = [dir_num, flower_name, num_images]
    map_sorted.append(row)
    
    # get image sizes for every image
    for image in os.listdir(target_directory):
        target_image = os.path.join(target_directory, image)
        image = Image.open(target_image)
        image_array = np.array(image)
        # get size of each image
        image_height = image_array.shape[0]
        image_width = image_array.shape[1]
        image_stats.append([dir_num, flower_name, image, 
                            image_width, image_height])

# create some handy dataframes
class_matrix = pd.DataFrame(data=map_sorted)
class_matrix = class_matrix.rename(mapper={0: "dir_num", 
                                           1: "flower_name", 
                                           2: "num_images"}, axis=1)

class_matrix["mean_num_images"] = class_matrix["num_images"].mean()
class_matrix["mean_num_images"] = class_matrix["mean_num_images"].astype(int)
class_matrix.head(5)

df_image_details = pd.DataFrame(data=image_stats)
df_image_details = df_image_details.rename(mapper={0: "dir_num", 
                                               1: "flower_name", 
                                               2: "image_file", 
                                               3: "width", 
                                               4: "height"}, axis=1)

# TODO: extract number of images by class into list; for use in DataLoader weighting
# class_sample_counts = class_matrix["num_images"].values.tolist()


In [ ]:
# see distribution of training data
labels = class_matrix["flower_name"]
values = class_matrix["num_images"]
means = class_matrix["mean_num_images"]

plt.figure(figsize=(18,4))
plt.title("# of images per flower")
ax = sns.barplot(labels, values, alpha = 0.9)
plt.bar(labels, values)
sns.lineplot(labels, means, color='#bc5145', 
             label=f'mean = {class_matrix["mean_num_images"].max()}')
plt.xticks(labels, rotation=90)
ax.set(xlabel='flower', ylabel='# of images')
ax.legend()
plt.show()

In [ ]:
df_image_stats = df_image_details.loc[:, ["flower_name", "width", "height"]]

In [ ]:
df_image_stats = df_image_stats.groupby("flower_name").describe()
df_image_stats.head()

In [ ]:
# finding:
# all images are at least 500px X 500px

In [ ]:
# define transformations and datasets
# important: parameterize final image size; pretrained models use different sizes

# Training data - gets augmentation

data_transforms_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(35),
    transforms.CenterCrop(size=image_size_required),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean_train_imgs, std_train_imgs)
    ])

# Validation data - no augmentation
data_transforms_valid = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(size=image_size_required),
    transforms.ToTensor(),
    transforms.Normalize(mean_train_imgs, std_train_imgs)
    ])

# Test data - no augmentation
data_transforms_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(size=image_size_required),
    transforms.ToTensor(),
    transforms.Normalize(mean_train_imgs, std_train_imgs)
    ])


# Load the datasets with ImageFolder
train_datasets = torchvision.datasets.ImageFolder(root=train_dir, transform=data_transforms_train)
valid_datasets = torchvision.datasets.ImageFolder(root=valid_dir, transform=data_transforms_valid)


# define dataloaders; note: shuffle=True, can't be used with sampling
train_loader = torch.utils.data.DataLoader(train_datasets, 
                                           batch_size=batch_size, 
                                           num_workers=num_workers, 
                                           shuffle=True)
                                           # rebalancing train happens here

valid_loader = torch.utils.data.DataLoader(valid_datasets, 
                                           batch_size=batch_size,  
                                           num_workers=num_workers)

# workaround- no test data provided in challenge, but must check test for scoring
# we'll create a conditional test and create loaders only if `test` data exists
if test_data_exists:
    test_datasets = torchvision.datasets.ImageFolder(root=test_dir, 
                                                     transform=data_transforms_test)
    
    test_sampler = RandomSampler(test_datasets, replacement=False)
    
    test_loader = torch.utils.data.DataLoader(test_datasets, 
                                              batch_size=batch_size, 
                                              sampler=test_sampler, 
                                              num_workers=num_workers)

# summarize input we have to work with
print("# of training images= ", len(train_datasets))
print("# of validation images = ", len(valid_datasets))
print("training data # of batches = ", len(train_loader))
print("validation data # of batches = ", len(valid_loader))

if test_data_exists:
    print("# of test images = ", len(test_datasets))
    print("test data # of batches = ", len(test_loader))
else:
    print("test data - none available - skipping")


### Visualize training data

In [ ]:
# create helper function to un-normalize and display an image

def imageshow_numpy(image_numpy):
    """
    Convert numpy image array to matplotlib format (fix color channel order), 
    normalize image with std & mean from training data & display image
    Function named to distinguish from matplotlib imshow
    
    :param image: numpy array representing image
    :return converted image
    """
    # PyTorch assumes color channel first dimension
    # but matplotlib assumes color channel third dimension; need to swap
    image = np.transpose(image_numpy, (1, 2, 0))
    
    # normalize images based on training data std and mean (given above)
    image = std_train_imgs * image + mean_train_imgs
    
    # clip to range to suppress warnings
    image = np.clip(image, 0, 1)
    
    plt.imshow(image)
    
    return image
    

In [ ]:
# print sample of a batch of training images
image_iterator = iter(train_loader)
images, labels = image_iterator.next()
images = images.numpy() # convert images to numpy for display

# plot the images with labels in the batch
fig = plt.figure(figsize=(25, 16))

for image_id in np.arange(1, 21, 1):
    ax = fig.add_subplot(4, 20/4, image_id, xticks=[], yticks=[])
    imageshow_numpy(images[image_id])
    ax.set_title(cat_to_name[labels[image_id]])  # label with cat # and name

### Humans - STOP Here
*look at your data_transforms; 

**intuitively, could you identify these flowers as displayed above based on zoom/crop + other transforms?**

*if not - adjust your data transformations*

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

### import pretrained model

In [ ]:
def download_pretrained_model(model_name):
    """
    Downloads pretrained model from torchvision
    :param model_name: name of pretrained model
    :type model_name: str
    
    :return pytorch model using chosen architecture, ready to use or modify
    """
    if model_name == 'vgg16':
        model = models.vgg16(pretrained=True)
    elif model_name == 'densenet121':
        model = models.densenet121(pretrained=True)
    elif model_name == 'densenet161':
        model = models.densenet161(pretrained=True)
    elif model_name == 'resnet152':
        model = models.resnet152(pretrained=True)
    else:
        print(f"{ model_name } is NOT implemented in this notebook."
              "The following models are available: vgg16, densenet121,or resnet152")
    
    # append more explicit model_name for later use
    model.model_name = model_name
    
    return model

pretrained_model = download_pretrained_model(chosen_model)

print(pretrained_model, "\n\n", pretrained_model.model_name)

### define custom classifier 

In [ ]:
# Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout

# ACTION - UPDATE
# class Sequential(nn.Module):
#    def __init__(self, n_inputs, n_classes):
#        super(Sequential, self).__init__()
#        self.fc1 = nn.Linear(n_inputs, int(hidden_layers))
#        self.dropout_20 = nn.Dropout(p=.2)
#        self.fc2 = nn.Linear(int(hidden_layers), n_classes)
        
#        # Zero Dropout layer      
#        self.dropout_0 = nn.Dropout(p=0)
        
#        # Output layer - to give probabilities per class
#        self.out = nn.Softmax(dim = 1)
#        # ACTION:  self.out = nn.LogSoftmax(dim = 1)
    
#    def forward(self, x):
        # add sequence of convolutional and max pooling layers -- ACTION - no convolutional or pooling is here.  FIX!
#        x = self.fc1(x)
#        x = self.dropout_20(F.relu(x))
#        x = self.fc2(x)
#        x=self.out(x)
#        return x

def create_custom_classifier(num_in_features, 
                             hidden_layers, 
                             num_out_features=num_classes):
    """
    Function that defines our custom classifier
    :param num_in_features: # of elements in next-to-last layer of pretrained network
    :param hidden_layers: 
    :type hidden_layers: boolean
    :param num_out_features: number of classifications to make; should equal number of possible classes

    :return customized nn.Sequential classifier
    """
    custom_classifier = nn.Sequential()
    
    if hidden_layers == "Zero":
        custom_classifier.add_module('fc0', nn.Linear(num_in_features, 102))

    elif hidden_layers == "Manual":
            # add layer, matching # of input features to # of the first hidden layer
        custom_classifier.add_module('fc0', nn.Linear(num_in_features, 512))
        custom_classifier.add_module('relu0', nn.ReLU())
        custom_classifier.add_module('drop0', nn.Dropout(.6))
        custom_classifier.add_module('fc1', nn.Linear(512, 256))
        custom_classifier.add_module('relu1', nn.ReLU())
        custom_classifier.add_module('drop1', nn.Dropout(.5))
        custom_classifier.add_module('output', nn.Linear(256, num_out_features))
        
    else:
        # add layer, matching # of input features to # of the first hidden layer
        custom_classifier.add_module('fc0', nn.Linear(num_in_features, hidden_layer_sizes[0][0]))
        custom_classifier.add_module('relu0', nn.ReLU())
        custom_classifier.add_module('drop0', nn.Dropout(.6))
        custom_classifier.add_module('relu1', nn.ReLU())
        custom_classifier.add_module('drop1', nn.Dropout(.5))
        for i, layer_group in enumerate(hidden_layer_sizes):
            custom_classifier.add_module('fc'+str(i+1), 
                                         nn.Linear(layer_group[0], 
                                                   layer_group[1]))
            custom_classifier.add_module('relu'+str(i+1), nn.ReLU())
            custom_classifier.add_module('drop'+str(i+1), nn.Dropout(.5))
        custom_classifier.add_module('output', nn.Linear(layer_group[1], num_out_features))

    return custom_classifier

In [ ]:
# TODO: Build and train your network

def freeze_layers(model):
    """
    Function freezes layers so weights and biases are not updated in a model
    required frequently, so make a function
    :param model: a pytorch model we want to freeze
    
    : return frozen model
    """
    for param in model.parameters():
        param.requires_grad = False
    
    return model


def setup_customize_model(pretrained_model, num_classes):
    """
    Freezes layers and replaces classifiers for predefined torchvision models
    examines each model, gets # of input/output features in last layer
    so we can match those numbers up with the sizes in our custom classifier
    :param model: a pretrained model that has already been downloaded
    :param num_classes: the number of output classes for classification
    :return model, optimizer, criterion - three neural net objects needed for training/evaluation
    """
    
    # ACTION - CONDENSE THIS IF SWITCH TO IN [LIST OF CASES]
    model = pretrained_model
    
    # important notes
    # 1- The last layer does  classification, but the NAME of the last layer differs between torchvision models
    # so we need an IF switch to properly replace the last layer
    # 2- get in_features of last layer as a parameter; and feed that to our custom classifier
    # 3- order of operations:  important to freeze the model before replacing classifier
    
    if model.model_name == 'vgg16':
        model.classifier_layer_name = 'classifier'
        n_inputs = model.classifier[0].in_features
        n_outputs = model.classifier[0].out_features
        model = freeze_layers(model)  # freeze layers
        
        # replace classifier
        model.classifier = create_custom_classifier(n_inputs,
                                                    hidden_layers,
                                                    num_classes)
        criterion = nn.NLLLoss()
        optimizer_target = model.classifier.parameters()

    elif model.model_name == 'densenet121':
        model.classifier_layer_name = 'classifier'
        n_inputs = model.classifier.in_features
        n_outputs = model.classifier.out_features
        model = freeze_layers(model)  # freeze layers
        
        # replace classifier
        model.classifier = create_custom_classifier(n_inputs,
                                                    hidden_layers,
                                                    num_classes)
        criterion = nn.CrossEntropyLoss()
        optimizer_target = model.classifier.parameters()
        
    elif model.model_name == 'densenet161':
        model.classifier_layer_name = 'classifier'
        n_inputs = model.classifier.in_features
        n_outputs = model.classifier.out_features
        model = freeze_layers(model)  # freeze layers
        
        # replace classifier
        model.classifier = create_custom_classifier(n_inputs,
                                                    hidden_layers,
                                                    num_classes)
        criterion = nn.CrossEntropyLoss()
        optimizer_target = model.classifier.parameters()

    elif model.model_name == 'resnet152':
        model.classifier_layer_name = 'fc'
        n_inputs = model.fc.in_features
        n_outputs = model.fc.out_features
        model = freeze_layers(model)  # freeze layers
        
        # replace classifier; note different name
        model.fc = create_custom_classifier(n_inputs,
                                            hidden_layers,
                                            num_classes)
        criterion = nn.CrossEntropyLoss()
        optimizer_target = model.fc.parameters()

    # setup criterion and optimizer
    # notes: CrossEntropyLoss can be helpful when unbalanced training set
    # https://pytorch.org/docs/stable/nn.html
        
    # setup optimizer
    # option below required for udacity
    # optimizer = optim.Adam(filter(lambda p: p.requires_grad, 
    #                       model.parameters()), 
    #                       lr=learning_rate)

    optimizer = optim.Adadelta(model.parameters())
    
    return model, optimizer, criterion 
    

model, optimizer, criterion = setup_customize_model(pretrained_model,
                                                   num_classes)

model

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} - total possible parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,}     training parameters will be used.")

model.total_trainable_params = total_trainable_params

### map classes to indices and vice versa, so we can save as part of model

In [ ]:
model.class_to_idx = train_datasets.class_to_idx
model.idx_to_class = {
    idx: class_
    for idx, class_ in cat_to_name
}

list(model.class_to_idx.items())[:20]

In [ ]:
list(model.idx_to_class.items())[:10]

In [ ]:
print("model name we are using is: ", model.__class__.__name__)
print("note that this name, does not include the version number - so not as useful as manually appended name above")

### build network - loss & optimizer

In [ ]:
# ACTION: Build and train your network

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model = model.to('cuda')

# use torch.summary
summary(model, input_size = (3, image_size_required, image_size_required), batch_size = batch_size)

### train network

In [ ]:
def train(model,
          optimizer,
          criterion,
          train_loader,
          valid_loader,
          global_saved_file_name,
          n_epochs,
          ):
    # number of epochs to train; defined in global constants @top
    n_epochs = n_epochs
    # setup parameters to stop training if no improvement
    epochs_no_improve = 0
    epochs_noprogress_limit = 10

    valid_loss_min = np.Inf  # track change in validation loss

    # all scheduler args defined in global constants
    scheduler = MultiStepLR(optimizer,
                            milestones,
                            gamma)

    # track history of results for plotting
    history = []
    
    # to help saving state_dict, and work around different layer names in model
    # configure variable to match final layer name (e.g. classifier or fc)
    if model.classifier_layer_name == 'classifier':
        classifier_layer = model.classifier
    if model.classifier_layer_name == 'fc':
        classifier_layer = model.fc
    
    status_message = (f"Training started on { chosen_model } model; output to { global_saved_file_name }")
    slack_message(status_message, [], channel_id)
    
    for epoch in range(1, n_epochs + 1):

        # check steps to learning rate
        scheduler.step()

        # keep track of LOSS and ACCURACY for both training and validation
        train_loss = 0
        valid_loss = 0

        train_acc = 0
        valid_acc = 0

        ###################
        # train the model #
        ###################
        model.train()
        for data, target in train_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
                model = model.to('cuda')
                
            # clear the gradients of all optimized variables
            optimizer.zero_grad()

            # forward pass: compute predicted outputs by passing inputs to the model

            # predicted outputs will be log probabilities
            output = model(data)

            # if inception_v3 use: output, aux = model(data)

            # calculate the batch loss
            loss = criterion(output, target)
            
            # backward pass
            # compute gradient of the loss; then update parameters
            loss.backward()
            optimizer.step()
            
            # update training loss
            train_loss += loss.item() * data.size(0)

            # Estimate accuracy by finding max log probability
            _, prediction = torch.max(output, dim=1)
            correct_tensor = prediction.eq(target.data.view_as(prediction))

            # Must convert correct_tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))

            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

        ######################
        # validate the model #
        ######################
        model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if train_on_gpu:
                data, target = data.cuda(), target.cuda(), 
                model = model.to('cuda')

            # forward pass: compute predicted outputs by passing inputs to the model
            # IMPORTANT - because we are using inceptionv3, the results are returned as a tuple that has to be unpacked per
            # https://github.com/pytorch/vision/issues/302
            # or we will get this error:  `AttributeError: 'tuple' object has no attribute 'log_softmax'`
            # , aux

            output = model(data)

            # calculate the batch loss
            loss = criterion(output, target)

            # update average validation loss
            valid_loss += loss.item() * data.size(0)

            # Calculate validation accuracy
            _, prediction = torch.max(output, dim=1)
            correct_tensor = prediction.eq(target.data.view_as(prediction))
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))

            # Multiply average accuracy times the number of examples
            valid_acc += accuracy.item() * data.size(0)

        # calculate average losses
        train_loss = train_loss / len(train_loader.dataset)
        valid_loss = valid_loss / len(valid_loader.dataset)

        # Calculate average accuracy
        train_acc = train_acc / len(train_loader.dataset)
        valid_acc = valid_acc / len(valid_loader.dataset)

        # print training/validation statistics
        print(f"epoch: {epoch}\t"
              f"Training Loss:  {train_loss:.6}\t\t"
              f"Validation Loss:  {valid_loss:.6}\n\t\t"
              f"Training Accuracy:  {train_acc:.6}\t\t"
              f"Validation Accuracy:  {valid_acc:.6}"
              )

        history.append([epoch, train_loss, valid_loss, train_acc, valid_acc])

        # save model if validation loss has decreased
        if valid_loss <= valid_loss_min:

            print(
                f"\t\tValidation loss decreased {valid_loss_min:.6} --> {valid_loss:.6} saving model...\n")
            current_state = {'model': model,
                             'model_name': model.model_name,
                             'state_dict': model.state_dict(),
                             'optimizer': optimizer.state_dict(),
                             'classifier': classifier_layer,  # can be model.classifier or model.fc
                             'epoch': epoch + 1,
                             'batch_size': batch_size,
                             'class_to_idx': model.class_to_idx,
                             'idx_to_class': model.idx_to_class
                             }

            # attach the optimizer to our model
            model.optimizer = optimizer
            model.epoch = epoch

            torch.save(current_state, global_saved_file_name)

            valid_loss_min = valid_loss
            
        else:
            epochs_no_improve += 1
            # Check early stopping condition
            if epochs_no_improve == epochs_noprogress_limit:
                stopping_message = (
                    f"Early stopping - "
                    f"training has not improved in "
                    f"{ epochs_noprogress_limit } epochs \n")
                print(stopping_message)
                slack_message(stopping_message, [], channel_id)

                # Load in the best model
                model = torch.load(global_saved_file_name)
                break
            else:
                continue

    print("\n ##########################################################")

    # dump training history to file
    history = pd.DataFrame(history, columns=['epoch',
                                             'train_loss',
                                             'valid_loss',
                                             'train_acc',
                                             'valid_acc'])
    
    history_filename = (global_saved_file_name + ".history.csv")
    history.to_csv(os.path.join(data_dir, history_filename))
    print("history.csv file saved")
    
    # post status to slack
    status_message = (f"epoch: { epoch } validation loss better; valid_acc = {valid_acc:.6} saving")
    slack_message(status_message, [], channel_id)
    status_message = (f"{ global_saved_file_name}history.v60.csv file saved")
    slack_message(status_message, [], channel_id)

    return model

In [ ]:
# switch to RAWNB convert to skip retraining on cpu only platforms
train(model,
      optimizer,
      criterion,
      train_loader,
      valid_loader,
      global_saved_file_name,
      n_epochs)


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model


# this function loads pretrained model AND state
def load_checkpoint(global_saved_file_name):
    """
    Loads previously trained neural network
    :param global_saved_file_name: name of checkpoint
    
    :return model ready for inference based on previous training
    """
    filename = global_saved_file_name
    print("\n LOADING checkpoint:")
    # Note: Input model & optimizer should be pre-defined.  This routine only updates their states.

    
    def confirm_checkpoint_exists(filename):
        if not os.path.isfile(filename):
            print("=> no checkpoint found at '{}'".format(filename))
            raise ValueError("Checkpoint file not found")
        else:
            return True
        
    
    def unpack_state_dict(checkpoint):
        """
        pytorch state_dict is complicated
        this function unpacks the dictionary and loads the values (state) into the model
        :param checkpoint: model that has been loaded into pytorch
        """
        if 'state_dict' not in checkpoint:
            raise ValueError("No `state_dict` exists in the saved checkpoint file")
        
        else:
            print("`state_dict` exists in checkpoint")

            print("iterating through state_dict keys")
            # state_dict = checkpoint['state_dict']
            unpacked_state_dict = OrderedDict()
            for k, v in checkpoint['state_dict'].items():
                name = k[7:]  # remove 'module.' of dataparallel
                unpacked_state_dict[name] = v

            model.load_state_dict(unpacked_state_dict, 
                                  strict=False)         # strict False required
        return True

    confirm_checkpoint_exists(global_saved_file_name)
    
    # load model and optimizer state
    print("=> loading checkpoint '{}' \n".format(filename))
    checkpoint = torch.load(filename, map_location=device)
    print(checkpoint.keys())

    model = checkpoint['model']
    
    # unpack nested state using function & then load it
    unpack_state_dict(checkpoint)
    model.load_state_dict(checkpoint['state_dict'])
    
    # load remaining model keys
    model.model_name = checkpoint['model_name']
    model.optimizer = checkpoint['optimizer']
    model.classifier = checkpoint['classifier']    
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_class = checkpoint['idx_to_class']
    model.epoch = checkpoint['epoch']
    model.batch_size = checkpoint['batch_size']
    
    # freeze all parameters - we're doing inference next
    for param in model.parameters():
        param.requires_grad = False

    print("model_state loaded")

    return model
    

In [ ]:
model = load_checkpoint(global_saved_file_name)

try:
   model.eval()
   print("model in evaluation mode")
except AttributeError as error:
   print(error)



# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def process_image(process_image_path):
    """
    Prepares a PIL image for our pytorch model
    Applies same scaling, crop, normalization as training images
    Uses pytorch transforms instead of manual cropping - easier to follow
    
    :param image_path: path to PIL image
    :type image_path: string
    
    :return tensor array representing the image
    """
    
    # Process a PIL image for use in a PyTorch model
    # tensor.numpy().transpose(1, 2, 0)    
    if not os.path.isfile(process_image_path) :
        print("path error in process image function")
    
    image = Image.open(process_image_path)
    
    # use pytorch transforms, much cleaner
    preprocess = transforms.Compose([
                        transforms.Resize(256),
                        transforms.CenterCrop(size=image_size_required),
                        transforms.ToTensor(),
                        transforms.Normalize(mean_train_imgs, std_train_imgs)
                        ])
    
    image_tensor = preprocess(image)  # note transformat.ToTensor() step
    
    return image_tensor


In [ ]:
def imageshow_tensor(image_tensor, ax=None, title=None):
    """
    Display an image passed in as a tensor
    :param image_tesnsor: tensor representing and image; 
    channels in order of (color, height, width)
    : return display and return matplotlib object, and PIL image
    """

    if ax is None:
        fig, ax = plt.subplots()

    # Set the color channel as the third dimension
    image = image_tensor.numpy().transpose((1, 2, 0))

    # Reverse the preprocessing steps
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Clip the image pixel values
    image = np.clip(image, 0, 1)

    ax.imshow(image)
    plt.axis('off')

    return ax, image


In [ ]:
image_2 = process_image(valid_dir + "/10/image_07094.jpg")
image_2.shape

In [ ]:
imageshow_tensor(image_2, ax=None);

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
# write function to predict class of input image
from torch.autograd import Variable

def predict(image_path, model, topk=5):
    """
    Predict classification of an image using a trained model
    :param image_path: path to image being tested
    :type image_path: string reprsenting path to the image
    :param model: pretrained neural network
    :param topk: number of top predictions to return
    :type topk: int
    
    :return 
        img_tensor.cpu().squeeze() - tensor of image
        top_p: (list) float probabilities of top(int) classes in order
        top_classes: (int) class numbers of top(int) classes, in order
        real_class: (int) class number of the actual class
        real_class_name: (str) name of the actual class
    """
    
    gc.collect()
    
    # mitigate path issues with different systems with os.sep
    real_class = image_path.split(os.sep)[-2]

    # Convert image to pytorch tensor
    image_tensor = process_image(image_path)

    if train_on_gpu:
        image_tensor = image_tensor.view(1, 3, 224, 224).cuda()
        model = model.to('cuda')
    else:
        image_tensor = image_tensor.view(1, 3, 224, 224)

    # set model to eval mode
    model.eval()

    inputs = Variable(image_tensor)
    logits = model.forward(inputs)
    probabilities = F.softmax(logits, dim=1)

    topk = probabilities.cpu().topk(topk)

    real_class_name = ''.join([tup[1] for tup in list(model.idx_to_class.items()) if tup[0] == int(real_class)])


    top_probabilities, classes = (e.data.numpy().squeeze().tolist() for e in topk)
    top_p = top_probabilities
    class_names = train_datasets.classes
    top_classes = [str(cat) + str(" : ") + dict(cat_to_name)[cat] for cat in classes]

    # PRINT RESULTS FOR EACH VARIABLE
    print("real class: ", real_class)
    print("real class name: ", real_class_name)
    print("top classes & flower names = ", top_classes)
    print("top_p probabilities = ", top_p)

    return image_tensor.cpu().squeeze(), real_class, real_class_name, top_classes, top_p

In [ ]:
# another predict single image, manually selected
image_3_path = "47\image_04957.jpg"

image_3_path = os.path.join(valid_dir, image_3_path)
print(image_3_path)
#image_3_path = "flower_data/valid/47/image_04957.jpg"
print(os.path.exists(image_3_path))
      
predict(image_3_path, model, topk=5)

In [ ]:
def get_random_image():
    """
    Select and returns one random test image from the validation directory
    use os.path.join to protect against os-specific path issues
    :param none - no input, uses global vars for path
    :return image_path: path to image (jpg, etc.)
    """
    
    dir_choice = str(np.random.choice(os.listdir(valid_dir)))
    dir_path = os.path.join(valid_dir, dir_choice)
    files = os.listdir(dir_path)
    # select a random file from directory; protect vs empty range
    if len(files) >= 2:
        index = random.randint(1, (len(files)-1))  # start / end
    else:
        index = 0
    
    image_path = os.path.join(dir_path, files[index])
    
    # uncomment this section to look at details in this dir
    # print("class = ", path.split(os.sep)[-1])
    # print("number of files in class = ", len(files))
    # print("index in class chosen = ", index)
    # print("image path = ", image_path)
    # print("image name = ", image_path.split(os.sep)[-1])
    # for i in range(len(files)):
    #    print("index chosen =", len(files), "  index = ", i, "  image path=", path + files[i])
        
    return image_path

get_random_image();


In [ ]:
# another predict single image, manually selected
image_3_path = "47\image_04957.jpg"

image_3_path = os.path.join(valid_dir, image_3_path)
print(image_3_path)
#image_3_path = "flower_data/valid/47/image_04957.jpg"
print(os.path.exists(image_3_path))
      
predict(image_3_path, model, topk=5)


In [ ]:
# predict single image - random

predict(get_random_image(), model, topk=5)


## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the validation accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# ACTION - cleanup
# TODO: Display an image along with the top 5 classes
# return img_tensor.cpu().squeeze(), real_class, real_class_name, top_classes, top_p

def display_prediction(image_path, model, topk_range):
    """
    Display image to test and classification predictions
    :param image_path: path to image file in directory
    :type image_path: str
    :param model: trained CNN
    :param topk_range: number of predictions to return (e.g. 5)
    :type topk_range: int
    
    :return True:  all functions must have return
    """

    # returned from predict(): img_tensor.cpu().squeeze(), top_p, top_classes, real_class
    # Get predictions
    img, real_class, real_class_name, top_classes, top_probabilities = predict(image_path, model, topk_range)
    # Convert results to dataframe for plotting
    print("type of top classes = ", type(top_classes), top_classes)
    result = pd.DataFrame({'p': top_probabilities}, index=top_classes)
    print(top_probabilities)
    print(result)

    # Show the image and a chart of top predictions (name & probabilities)
    plt.figure(figsize=(16, 5))
    ax = plt.subplot(1, 2, 1)
    ax, img = imageshow_tensor(img, ax=ax)
    
    title_text = list([tup for tup in list(model.idx_to_class.items()) if tup[0] == int(real_class)])
    # list(title_text)
    # Set title to be the actual class
    ax.set_title(f"{title_text[0][0]} / {title_text[0][1]}", size=20)

    ax = plt.subplot(1, 2, 2)
    # Plot a bar plot of predictions
    result.sort_values('p')['p'].plot.barh(color='blue', edgecolor='k', ax=ax)
    
    plt.xlim(0, 1)
    plt.xlabel('Predicted Probability')
    plt.tight_layout()
    
    return True

In [ ]:
display_prediction(get_random_image(), model, 5)

In [ ]:
for flower in range(1,21):
    display_prediction(get_random_image(), model, 5)
    print("\n")
    